In [ ]:
# importing various required libraries and modules

import os  # for dealing with files and directories
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for visualization and also for reading & displaying images
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator # for Data Augmentation
import keras

### Preparing the Data

In [ ]:
# path to the directory where the images are placed
directory_path='../input/whatsapp-image-dataset/'

# this is the dictionary which will store the labels for our classes
class_dict={'paper':0,'poster':1,'meme':2,'screenshot':3,'family':4,'others':5}

In [ ]:
# creating a list of categories (different classes within the dataset)
#  and filenames (the .jpg name of the files)

categories=[]
filenames=os.listdir(directory_path)
for filename in filenames:
    category=filename.split(' ')[0]
    categories.append(class_dict[category])

In [ ]:
# creating a dataset
Dataset=pd.DataFrame({'filename':filenames,'label':categories})

In [ ]:
# visualizing the distribution of classes in Dataset

count_series=Dataset['label'].value_counts().sort_index()
sns.barplot(x=count_series.index, y=count_series)

In [ ]:
# splitting into train and test dataframes
train,test=train_test_split(Dataset,test_size=0.2,random_state=42)

In [ ]:
# visualizing the distribution of classes in train dataset

count_series_train=train['label'].value_counts().sort_index()
sns.barplot(x=count_series_train.index, y=count_series_train)

In [ ]:
# visualizing the distribution of classes in test dataset

count_series_test=test['label'].value_counts().sort_index()
sns.barplot(x=count_series_test.index, y=count_series_test)

#### Splitting train to train_df and valid_df

In [ ]:
# splitting into train_df and valid_df dataframes
train_df,valid_df=train_test_split(train,test_size=0.2,random_state=42)

In [ ]:
# visualizing the distribution of classes in test dataset

count_series_train_df=train_df['label'].value_counts().sort_index()
sns.barplot(x=count_series_train_df.index, y=count_series_train_df)

In [ ]:
# visualizing the distribution of classes in test dataset

count_series_valid_df=valid_df['label'].value_counts().sort_index()
sns.barplot(x=count_series_valid_df.index, y=count_series_valid_df)

In [ ]:
count_series_train_df.sum()

In [ ]:
count_series_test.sum()

In [ ]:
count_series_valid_df.sum()

### Importing the model and making it ready for transfer learning

In [ ]:
from keras.applications.mobilenet import MobileNet, preprocess_input

In [ ]:
base_model=MobileNet(weights='imagenet',include_top=False)

In [ ]:
# including top layer having number of neurons equal to the total number of class types

avg=keras.layers.GlobalAveragePooling2D()(base_model.output)
output=keras.layers.Dense(6, activation='softmax')(avg) # because the number of classes are 6
model=keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
# freezing all the layers of the model except the top one
base_model.trainable = False

#### Data Generators and Augmentation

In [ ]:
class_dict_inv=new_dict = dict(zip(class_dict.values(), class_dict.keys()))

# changed the label from numerical to categorical as flow_from_dataframe takes up strings instead of labels for classes
train_df['label']=train_df['label'].replace(class_dict_inv)

In [ ]:
valid_df['label']=valid_df['label'].replace(class_dict_inv)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    zoom_range=0.1,
    rotation_range=10,
    brightness_range=[0.5,1.2],
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
    train_df,
    directory_path,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=valid_datagen.flow_from_dataframe(
    valid_df,
    directory_path,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_eval_generator=test_datagen.flow_from_dataframe(
    test,
    directory_path,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

### Getting ready to train the model

In [ ]:
model.summary()

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.0001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#### Defining the Callbacks

In [ ]:
checkpoint_cb=keras.callbacks.ModelCheckpoint('project_cnn.h5',save_best_only=True)

In [ ]:
early_stopping_cb=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [ ]:
learning_rate_cb=keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)

## training process

### first run

In [ ]:
history= model.fit(
    train_generator, 
    epochs=25,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
# model validation accuracy reaches 95.07 %..saving the model and then preparing for unfreezing the next layer
model.save('project_cnn_run1.h5')

### second run

In [ ]:
# let's have a look of the current trainable layers

layers = [(layer.name, layer.trainable) for layer in model.layers]

pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable']).tail(20) #picking the last 20 entries of the dataframe

In [ ]:
# As expected only the dense layer and pooling layer defined by us are presently unfreezed

In [ ]:
# Now, we will unfreeze till conv_dw_13...
# Thus only the last block of convolutional layers will be unfreezed

model.trainable = True

set_trainable = False
for layer in model.layers:
    if layer.name in ['conv_dw_13']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# checking whether the desired layers have been unfreezed
layers = [(layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable']).tail(20)     

In [ ]:
# compiling the model before the second run
# choose a small learning rate so as to keep the weigths of the freshly unfreezed layer safe

optimizer=keras.optimizers.Adam(lr=0.00001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history= model.fit(
    train_generator, 
    epochs=100,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
model.save('project_cnn_run2.h5')

In [ ]:
# max accuracy reaching 95.55% with the current number of unfreezed layers
train_df.to_csv('./train_df.csv')

In [ ]:
test.to_csv('./test.csv')

In [ ]:
valid_df.to_csv('./valid_df.csv')

### third run

In [ ]:
# Now, we will unfreeze till conv_dw_13...
# Thus only the last block of convolutional layers will be unfreezed
def set_trainable(layer_name,no_of_last_rows):
    model.trainable = True

    set_trainable = False
    for layer in model.layers:
        if layer.name in [layer_name]:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    # checking whether the desired layers have been unfreezed
    layers = [(layer.name, layer.trainable) for layer in model.layers]
    return pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable']).tail(no_of_last_rows)  

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.000005)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history= model.fit(
    train_generator, 
    epochs=100,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
model.save('project_cnn_day2_run3.h5')

### fourth run

In [ ]:
model.trainable = True

set_trainable = False
for layer in model.layers:
    if layer.name in ['conv_dw_9']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# checking whether the desired layers have been unfreezed
layers = [(layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable']).tail(40)  

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.000001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history= model.fit(
    train_generator, 
    epochs=100,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
model.evaluate(test_eval_generator)

In [ ]:
model.save('project_cnn_day2_run4.h5')

### fifth + sixth run

In [ ]:
set_trainable('conv_dw_7',50)

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.0000001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history= model.fit(
    train_generator, 
    epochs=100,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
model.evaluate(test_eval_generator)

In [ ]:
model=keras.models.load_model('./project_cnn_day2_run4.h5')

In [ ]:
model.evaluate(test_eval_generator)

In [ ]:
test['label']=test['label'].replace(class_dict_inv)

In [ ]:
# testing on the test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_dataframe(
    test,
    directory_path,
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224,224),
    batch_size=32,
    shuffle=False
)

In [ ]:
y_pred=model.predict(test_generator)

In [ ]:
y_pred

In [ ]:
y_pred=pd.DataFrame(np.argmax(y_pred,axis=1)).replace(class_dict_inv).values

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(test['label'],y_pred)

## Reloading the saved model

In [ ]:
train_df=pd.read_csv('../input/csv-files/train_df.csv')

In [ ]:
test=pd.read_csv('../input/csv-files/test.csv')

In [ ]:
valid_df=pd.read_csv('../input/csv-files/valid_df.csv')

In [ ]:
train_df

In [ ]:
model=keras.models.load_model('../input/saved-models/project_cnn_run3.h5')

In [ ]:
history=model.fit( 
    train_generator, 
    epochs=1,
    validation_data=valid_generator,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_dataframe(
    test,
    directory_path,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
model.evaluate(test_generator)

### Base Loss and Acuuracy
[0.23634479939937592, 0.9377382397651672]

In [ ]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_gen.flow_from_dataframe(
    test, 
    directory_path, 
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32,
)

In [ ]:
model1=keras.models.load_model('../input/saved-models/project_cnn_final_model.h5')

In [ ]:
model=keras.models.load_model('../input/saved-models/project_cnn_day2_run4_9466.h5')

In [ ]:
model.evaluate(test_eval_generator)

In [ ]:
test_generator.reset()
y_pred=model.predict(test_generator)

In [ ]:
test.drop('class',axis=1,inplace=True)

In [ ]:
see_test=test

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
see_test['class']=pd.DataFrame(np.argmax(y_pred,axis=1)).replace(labels)

In [ ]:
y_true=see_test['label']
y_pred=see_test['class']

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_true,y_pred)

In [ ]:
train_generator.class_indices

In [ ]:
confusion_matrix(y_true,y_pred)

In [ ]:
confusion_matrix(y_true,y_pred)

In [ ]:
peek=(see_test[(see_test['label']=='screenshot')&(see_test['class']=='family')]).reset_index(drop=True)

plt.figure(figsize=(40,10))
for index, row in peek.iterrows():
  
    filename = row['filename']
    img = plt.imread(directory_path+filename)
    plt.subplot(3,1, index+1)
    plt.imshow(img)

plt.tight_layout()
plt.show()

In [ ]:
test.iloc[:32,1]

In [ ]:
y_pred

In [ ]:
model1.save('project_cnn_final_model.h5')

In [ ]:
model1.evaluate(test_eval_generator)

In [ ]:
model.evaluate(test_eval_generator)

In [ ]:
model1

In [ ]:
model

In [ ]:
import matplotlib.image as image
new_img = plt.imread('../input/new-image/eagle.jpg')
img = np.resize(new_img,(224, 224))

In [ ]:
filenames=os.listdir('../input/new-image')
new_test_df=pd.DataFrame({'filename':filenames})

In [ ]:
new_test_df

In [ ]:
new_gen = ImageDataGenerator(rescale=1./255)
new_generator = new_gen.flow_from_dataframe(
    new_test_df,
    '../input/new-image',
    class_mode=None,
    target_size=(224,224),
    batch_size=32,
    shuffle=False
)

In [ ]:
arr=model1.predict(new_generator)

In [ ]:
np.argmax(arr,axis=1)

In [ ]:
new_test_df['class']=pd.DataFrame(np.argmax(arr,axis=1)).replace(labels)

In [ ]:
new_test_df

In [ ]:
plt.figure(figsize=(12, 24))
for index, row in new_test_df.iterrows():
  
    filename = row['filename']
    category = row['class']
    img = plt.imread("../input/new-image/"+filename)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 24))
for index, row in new_test_df.iterrows():
  
    filename = row['filename']
    category = row['class']
    img = plt.imread("../input/new-image/"+filename)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )

plt.tight_layout()
plt.show()

### Converting the model to .tflite file

In [ ]:
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('project_cnn_9466.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model1)
tflite_model1 = converter.convert()

# Save the model.
with open('project_cnn_final.tflite', 'wb') as f:
  f.write(tflite_model1)

In [ ]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
convert_bytes(get_file_size('./project_cnn_9466.tflite'), "KB")

In [ ]:
# loading the tflite model and checking it's accuracy

interpreter = tf.lite.Interpreter(model_path ='./project_cnn_9466.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.get_input_details()

In [ ]:
interpreter.get_output_details()